# Intro to Spark #
## What is Spark?
Basic info about Apache Spark
## SparkContext: how you interact with your data
(explain enough so the following cell makes sense)

In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

## Resilient Distributed Datasets (RDD)
Now that we can interact with our data using a SparkSession/context, what happens to that data? We put it in RDDs!

In [2]:
flight_file = '../data/flights.csv'
txt = sc.textFile(flight_file)
print("We have {} flights!".format(txt.count()))

python_lines = txt.filter(lambda line: 'pdx' in line.lower())
print("Of these flights, {} involved PDX".format(python_lines.count()))

We have 12716 flights!
Of these flights, 270 involved PDX
